In [1]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\FER2013-augmented')
LABELS = os.listdir(PATH)

In [3]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/face-emotion-recognition"

Repository IdjiotSandwiches/face-emotion-recognition initialized!

In [4]:
K_SIZE = (21,21)
SIGMA = 3
THETA_RANGE = np.arange(0, np.pi, np.pi/32)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_32F
IMAGE_SIZE = (224,224)
N_COMPONENTS = 0.95
N_COMPONENTS_LOCAL = 16
BLUR = (5,5)

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [4]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\combined\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\combined\\labels.npy')

In [14]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
CALIB_SIZE = 0.7

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

calib_params = {
  'test_size': CALIB_SIZE,
  'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params, stratify=labels)
X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params, stratify=y_test)
# X_train, _, y_train, _ = train_test_split(
#     X_train, y_train, train_size=2000, stratify=y_train, random_state=42
# )

In [24]:
svc_model = SVC()

param_grid = {
    'C': [10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [3, 4],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(
    estimator=svc_model,
    param_grid=param_grid,
    scoring='accuracy',
    # n_jobs=4,
    cv=3,
    verbose=2
)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


: 

In [15]:
model_params = {
    'C': 20.0,
    # 'penalty': 'l2',
    'dual': False,
    'verbose': True,
    'max_iter': 5000
}

model = LinearSVC(**model_params).fit(X_train, y_train)
model = CalibratedClassifierCV(model, method='isotonic', cv='prefit')
model.fit(X_calib, y_calib)

predict = model.predict(X_test)
predict_proba = model.predict_proba(X_test)

metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
    'auc_score': roc_auc_score(y_test, predict_proba, multi_class='ovr', average='macro')
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}\nAUC Score: {metrics['auc_score']}")
report = classification_report(y_test, predict)
print(report)

mlflow.set_experiment("SVM_Model FER2013 augmented")
mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

with mlflow.start_run(run_name=f'SVM LinearSVC'):
    # mlflow.log_params(gabor_params)
    mlflow.log_params(model_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_param('image_size', IMAGE_SIZE)
    # mlflow.log_param('PCA_n_components', N_COMPONENTS)
    mlflow.log_param('gaussian_blur', BLUR)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path='SVM Model',
        input_example=X_train[:1]
    )

[LibLinear]Accuracy: 0.19464285714285715
Precision: 0.19169995383732086
Recall: 0.19464285714285712
F1 Score: 0.15800203516810069
AUC Score: 0.5304509223828421
              precision    recall  f1-score   support

       angry       0.17      0.05      0.08      1680
        fear       0.18      0.14      0.16      1680
       happy       0.20      0.07      0.10      1680
     neutral       0.20      0.60      0.29      1680
         sad       0.20      0.06      0.09      1680
    surprise       0.20      0.26      0.23      1680

    accuracy                           0.19     10080
   macro avg       0.19      0.19      0.16     10080
weighted avg       0.19      0.19      0.16     10080

🏃 View run SVM LinearSVC at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/19/runs/38f1db51d4444e8a8f75d05376a7d2f7
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/19


NameError: name 'FLOATING_POINT' is not defined